In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from functools import reduce

In [2]:
# FSVA

# Path folder dan output
data_folder = Path(r"../data/Data Raw/FSVA")
output_path = Path(r"../data/Data Clean")
output_path.mkdir(parents=True, exist_ok=True)

# Inisialisasi list penyimpanan data
all_dfs = []

# Loop file CSV
for file in sorted(data_folder.glob("*.csv")):
    tahun = int(file.stem)  # Ambil tahun dari nama file
    print(f"📄 Memproses file: {file.name}")
    
    df = pd.read_csv(file)

    # Tambahkan kolom Tahun
    df["Tahun"] = tahun

    df = df.rename(columns={"Komposit": "Kelompok IKP"})

    # Rapikan kolom wilayah
    df["Wilayah"] = (
        df["Wilayah"]
        .str.split(" - ").str[-1]  # Ambil nama kabupaten/kota
        .str.title()               # Kapitalisasi tiap awal kata
        .str.strip()
    )

    df = df.rename(columns={"Wilayah": "Kabupaten/Kota"})

    all_dfs.append(df)

# Gabung semua data
final_df = pd.concat(all_dfs, ignore_index=True)

# Susun urutan kolom sesuai keinginan
kolom_urutan = [
    "Kabupaten/Kota", "Kelompok IKP", "Tahun", "NCPR", "Kemiskinan (%)", "Pengeluaran Pangan (%)", 
    "Tanpa Listrik (%)", "Tanpa Air Bersih (%)", "Lama Sekolah Perempuan (tahun)", 
    "Rasio Tenaga Kesehatan", "Angka Harapan Hidup (tahun)", "Stunting (%)"
]

# Filter hanya kolom yang sesuai
final_df = final_df[kolom_urutan]

# Simpan hasil
final_df.to_csv(output_path / "fsva_clean.csv", index=False)

# Ringkasan
print("✅ Total baris:", len(final_df))
print("✅ Total wilayah unik:", final_df['Kabupaten/Kota'].nunique())
print("✅ Disimpan di:", output_path / "fsva_clean.csv")

📄 Memproses file: 2018.csv
📄 Memproses file: 2019.csv
📄 Memproses file: 2020.csv
📄 Memproses file: 2021.csv
📄 Memproses file: 2022.csv
📄 Memproses file: 2023.csv
📄 Memproses file: 2024.csv
✅ Total baris: 3598
✅ Total wilayah unik: 514
✅ Disimpan di: ..\data\Data Clean\fsva_clean.csv


In [3]:
# Path folder file
data_folder = Path("../data/Data Raw")
output_path = Path("../data/Data Clean")

In [4]:
# JUMLAH PENDUDUK MISKIN
# File yang ingin diproses
file_names = [
    "Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2016-2018.xlsx",
    "Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2019-2021.xlsx",
    "Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2022-2024.xlsx"
]

# List untuk menyimpan semua dataframe
all_dfs = []

# Baca dan ubah format setiap file
for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])  # 🔄 baca '-' sebagai NaN

    # Ubah dari wide ke long
    df_long = df.melt(
        id_vars="Nama Wilayah", 
        var_name="Tahun", 
        value_name="Jumlah Penduduk Miskin"
    )

    # Ekstrak angka tahun dari nama kolom
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

    all_dfs.append(df_long)

# Gabungkan semua file
final_df = pd.concat(all_dfs, ignore_index=True)

# Hapus baris provinsi (huruf kapital semua)
final_df = final_df[~final_df["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Ganti nama kolom
final_df = final_df.rename(columns={"Nama Wilayah": "Kabupaten/Kota"})

# Filter tahun >= 2018
final_df = final_df[final_df["Tahun"] >= 2018].reset_index(drop=True)

# 🛠 Pindahkan nilai dari 'Nduga *' ke 'Nduga', atau buat jika belum ada
for tahun in final_df["Tahun"].unique():
    nilai_nduga_star = final_df.loc[
        (final_df["Kabupaten/Kota"] == "Nduga *") & (final_df["Tahun"] == tahun),
        "Jumlah Penduduk Miskin"
    ]

    if nilai_nduga_star.empty:
        continue

    nilai = nilai_nduga_star.values[0]

    mask_nduga = (final_df["Kabupaten/Kota"] == "Nduga") & (final_df["Tahun"] == tahun)
    
    if mask_nduga.any():
        if pd.isna(final_df.loc[mask_nduga, "Jumlah Penduduk Miskin"]).values[0]:
            final_df.loc[mask_nduga, "Jumlah Penduduk Miskin"] = nilai
    else:
        final_df = pd.concat([
            final_df,
            pd.DataFrame({
                "Kabupaten/Kota": ["Nduga"],
                "Tahun": [tahun],
                "Jumlah Penduduk Miskin": [nilai]
            })
        ], ignore_index=True)

# ❌ Hapus baris tidak valid
final_df = final_df[~final_df["Kabupaten/Kota"].isin(["Nduga *", "KEP. BANGKA BELITUNG"])]

# 🧹 Prioritaskan baris yang punya nilai (non-NaN)
final_df = final_df.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Jumlah Penduduk Miskin"],
    ascending=[True, True, True]
)

# Hapus duplikat berdasarkan kombinasi Kab/Kota dan Tahun
final_df = final_df.drop_duplicates(
    subset=["Kabupaten/Kota", "Tahun"],
    keep="first"
).reset_index(drop=True)

# Simpan ke CSV
final_df.to_csv(output_path / "jumlah_penduduk_miskin_long.csv", index=False)

# Tampilkan ringkasan hasil
print("✅ Jumlah Kab/Kota unik:", final_df["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris akhir:", final_df.shape[0])

📄 Membaca: Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2016-2018.xlsx
📄 Membaca: Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2019-2021.xlsx
📄 Membaca: Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2022-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris akhir: 3598


In [5]:
#IPM

# Nama file IPM
file_names = [
    "[Metode Baru] Indeks Pembangunan Manusia, 2016-2018.xlsx",
    "[Metode Baru] Indeks Pembangunan Manusia, 2019-2021.xlsx",
    "[Metode Baru] Indeks Pembangunan Manusia, 2022-2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])  # konversi '-' jadi NaN

    # Wide ke long
    df_long = df.melt(
        id_vars="Nama Wilayah", 
        var_name="Tahun", 
        value_name="Indeks Pembangunan Manusia"
    )

    # Ekstrak angka tahun
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

    all_dfs.append(df_long)

# Gabungkan semua file
final_df = pd.concat(all_dfs, ignore_index=True)

# Filter tahun >= 2018
final_df = final_df[final_df["Tahun"] >= 2018].reset_index(drop=True)

# Hapus baris provinsi (huruf kapital semua)
final_df = final_df[~final_df["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Ubah nama kolom
final_df = final_df.rename(columns={"Nama Wilayah": "Kabupaten/Kota"})

# Bersihkan kabupaten/kota
final_df["Kabupaten/Kota"] = (
    final_df["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Hapus entri tidak valid
final_df = final_df[~final_df["Kabupaten/Kota"].isin(["KEP. BANGKA BELITUNG"])]

# Prioritaskan nilai yang tidak kosong jika ada duplikat
final_df = final_df.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Indeks Pembangunan Manusia"],
    ascending=[True, True, True]
)
final_df = final_df.drop_duplicates(
    subset=["Kabupaten/Kota", "Tahun"],
    keep="first"
).reset_index(drop=True)

# Simpan ke CSV
final_df.to_csv(output_path / "ipm_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", final_df["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", final_df.shape[0])

📄 Membaca: [Metode Baru] Indeks Pembangunan Manusia, 2016-2018.xlsx
📄 Membaca: [Metode Baru] Indeks Pembangunan Manusia, 2019-2021.xlsx
📄 Membaca: [Metode Baru] Indeks Pembangunan Manusia, 2022-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [6]:
# File AHH
file_names = [
    "Angka Harapan Hidup (AHH) Menurut Kabupaten_Kota dan Jenis Kelamin, 2016-2018.csv",
    "Angka Harapan Hidup (AHH) Menurut Kabupaten_Kota dan Jenis Kelamin, 2019-2021.csv",
    "Angka Harapan Hidup (AHH) Menurut Kabupaten_Kota dan Jenis Kelamin, 2022-2024.csv"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_csv(file_path, na_values=["-"])

    # Pastikan kolom nama wilayah konsisten
    df = df.rename(columns={df.columns[0]: "Nama Wilayah"})

    # Wide ke long
    df_long = df.melt(
        id_vars="Nama Wilayah", 
        var_name="Tahun", 
        value_name="Angka Harapan Hidup"
    )

    # Ekstrak tahun dari nama kolom
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

    all_dfs.append(df_long)

# Gabungkan semua
final_df = pd.concat(all_dfs, ignore_index=True)

# Filter tahun 2018 ke atas
final_df = final_df[final_df["Tahun"] >= 2018].reset_index(drop=True)

# Hapus provinsi (nama kapital semua)
final_df = final_df[~final_df["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Ubah nama kolom
final_df = final_df.rename(columns={"Nama Wilayah": "Kabupaten/Kota"})

# Bersihkan nama wilayah
final_df["Kabupaten/Kota"] = (
    final_df["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Hapus entri tidak valid
final_df = final_df[~final_df["Kabupaten/Kota"].isin(["KEP. BANGKA BELITUNG"])]

# Prioritaskan nilai valid (non-NaN) jika ada duplikat
final_df = final_df.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Angka Harapan Hidup"],
    ascending=[True, True, True]
)
final_df = final_df.drop_duplicates(
    subset=["Kabupaten/Kota", "Tahun"],
    keep="first"
).reset_index(drop=True)

# Simpan ke CSV
final_df.to_csv(output_path / "ahh_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", final_df["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", final_df.shape[0])

📄 Membaca: Angka Harapan Hidup (AHH) Menurut Kabupaten_Kota dan Jenis Kelamin, 2016-2018.csv
📄 Membaca: Angka Harapan Hidup (AHH) Menurut Kabupaten_Kota dan Jenis Kelamin, 2019-2021.csv
📄 Membaca: Angka Harapan Hidup (AHH) Menurut Kabupaten_Kota dan Jenis Kelamin, 2022-2024.csv
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [7]:
# Garis Kemiskinan
# File tunggal
file_path = data_folder / "Garis Kemiskinan Menurut Kabupaten_Kota, 2018-2024.xlsx"

# Baca file
print(f"📄 Membaca: {file_path.name}")
df = pd.read_excel(file_path, na_values=["-"])

# Rename kolom nama wilayah
df = df.rename(columns={df.columns[0]: "Nama Wilayah"})

# Wide ke long
df_long = df.melt(
    id_vars="Nama Wilayah", 
    var_name="Tahun", 
    value_name="Garis Kemiskinan"
)

# Ekstrak angka tahun
df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

# Filter tahun >= 2018
df_long = df_long[df_long["Tahun"] >= 2018].reset_index(drop=True)

# Hapus baris provinsi
df_long = df_long[~df_long["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Rename kolom
df_long = df_long.rename(columns={"Nama Wilayah": "Kabupaten/Kota"})

# Bersihkan nama kabupaten/kota
df_long["Kabupaten/Kota"] = (
    df_long["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# 🔁 Pindahkan nilai dari 'Nduga *' ke 'Nduga' jika diperlukan
for tahun in df_long["Tahun"].unique():
    nilai_nduga_star = df_long.loc[
        (df_long["Kabupaten/Kota"] == "Nduga *") & (df_long["Tahun"] == tahun),
        "Garis Kemiskinan"
    ]

    if nilai_nduga_star.empty:
        continue

    nilai = nilai_nduga_star.values[0]

    mask_nduga = (df_long["Kabupaten/Kota"] == "Nduga") & (df_long["Tahun"] == tahun)
    
    if mask_nduga.any():
        if pd.isna(df_long.loc[mask_nduga, "Garis Kemiskinan"]).values[0]:
            df_long.loc[mask_nduga, "Garis Kemiskinan"] = nilai
    else:
        # Jika belum ada baris 'Nduga', tambahkan
        df_long = pd.concat([
            df_long,
            pd.DataFrame({
                "Kabupaten/Kota": ["Nduga"],
                "Tahun": [tahun],
                "Garis Kemiskinan": [nilai]
            })
        ], ignore_index=True)

# Hapus entri tidak valid
df_long = df_long[~df_long["Kabupaten/Kota"].isin(["Nduga *", "KEP. BANGKA BELITUNG"])]

# Urutkan & hilangkan duplikat
df_long = df_long.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Garis Kemiskinan"],
    ascending=[True, True, True]
)
df_long = df_long.drop_duplicates(
    subset=["Kabupaten/Kota", "Tahun"],
    keep="first"
).reset_index(drop=True)

# Simpan ke CSV
df_long.to_csv(output_path / "garis_kemiskinan_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_long["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_long.shape[0])

📄 Membaca: Garis Kemiskinan Menurut Kabupaten_Kota, 2018-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [8]:
# File P1
file_names = [
    "Indeks Kedalaman Kemiskinan (P1) Menurut Kabupaten_Kota, 2016-2018.xlsx",
    "Indeks Kedalaman Kemiskinan (P1) Menurut Kabupaten_Kota, 2019-2021.xlsx",
    "Indeks Kedalaman Kemiskinan (P1) Menurut Kabupaten_Kota, 2022-2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])
    df = df.rename(columns={df.columns[0]: "Nama Wilayah"})

    # Wide ke long
    df_long = df.melt(
        id_vars="Nama Wilayah", 
        var_name="Tahun", 
        value_name="Indeks Kedalaman Kemiskinan"
    )

    # Ekstrak angka tahun
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

    all_dfs.append(df_long)

# Gabungkan semua
final_df = pd.concat(all_dfs, ignore_index=True)

# Filter tahun >= 2018
final_df = final_df[final_df["Tahun"] >= 2018].reset_index(drop=True)

# Hapus baris provinsi
final_df = final_df[~final_df["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Ganti nama kolom wilayah
final_df = final_df.rename(columns={"Nama Wilayah": "Kabupaten/Kota"})

# Bersihkan nama kab/kota
final_df["Kabupaten/Kota"] = (
    final_df["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Pindahkan nilai dari 'Nduga *' ke 'Nduga' jika kosong
for tahun in final_df["Tahun"].unique():
    nilai_nduga_star = final_df.loc[
        (final_df["Kabupaten/Kota"] == "Nduga *") & (final_df["Tahun"] == tahun),
        "Indeks Kedalaman Kemiskinan"
    ]
    if nilai_nduga_star.empty:
        continue
    nilai = nilai_nduga_star.values[0]
    mask_nduga = (final_df["Kabupaten/Kota"] == "Nduga") & (final_df["Tahun"] == tahun)
    if mask_nduga.any():
        if pd.isna(final_df.loc[mask_nduga, "Indeks Kedalaman Kemiskinan"]).values[0]:
            final_df.loc[mask_nduga, "Indeks Kedalaman Kemiskinan"] = nilai
    else:
        final_df = pd.concat([
            final_df,
            pd.DataFrame({
                "Kabupaten/Kota": ["Nduga"],
                "Tahun": [tahun],
                "Indeks Kedalaman Kemiskinan": [nilai]
            })
        ], ignore_index=True)

# Hapus entri tidak valid
final_df = final_df[~final_df["Kabupaten/Kota"].isin(["Nduga *", "KEP. BANGKA BELITUNG"])]

# Hapus duplikat
final_df = final_df.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Indeks Kedalaman Kemiskinan"],
    ascending=[True, True, True]
)
final_df = final_df.drop_duplicates(
    subset=["Kabupaten/Kota", "Tahun"],
    keep="first"
).reset_index(drop=True)

# Simpan hasil
final_df.to_csv(output_path / "p1_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", final_df["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", final_df.shape[0])

📄 Membaca: Indeks Kedalaman Kemiskinan (P1) Menurut Kabupaten_Kota, 2016-2018.xlsx
📄 Membaca: Indeks Kedalaman Kemiskinan (P1) Menurut Kabupaten_Kota, 2019-2021.xlsx
📄 Membaca: Indeks Kedalaman Kemiskinan (P1) Menurut Kabupaten_Kota, 2022-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [9]:
# File P2
file_names = [
    "Indeks Keparahan Kemiskinan (P2) Menurut Kabupaten_Kota, 2016-2018.xlsx",
    "Indeks Keparahan Kemiskinan (P2) Menurut Kabupaten_Kota, 2019-2021.xlsx",
    "Indeks Keparahan Kemiskinan (P2) Menurut Kabupaten_Kota, 2022-2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])
    df = df.rename(columns={df.columns[0]: "Nama Wilayah"})

    # Wide ke long
    df_long = df.melt(
        id_vars="Nama Wilayah", 
        var_name="Tahun", 
        value_name="Indeks Keparahan Kemiskinan"
    )

    # Ekstrak angka tahun
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

    all_dfs.append(df_long)

# Gabungkan semua
final_df = pd.concat(all_dfs, ignore_index=True)

# Filter tahun >= 2018
final_df = final_df[final_df["Tahun"] >= 2018].reset_index(drop=True)

# Hapus baris provinsi
final_df = final_df[~final_df["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Ganti nama kolom wilayah
final_df = final_df.rename(columns={"Nama Wilayah": "Kabupaten/Kota"})

# Bersihkan nama kab/kota
final_df["Kabupaten/Kota"] = (
    final_df["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# 🔁 Tangani kasus 'Nduga *' → 'Nduga'
for tahun in final_df["Tahun"].unique():
    nilai_nduga_star = final_df.loc[
        (final_df["Kabupaten/Kota"] == "Nduga *") & (final_df["Tahun"] == tahun),
        "Indeks Keparahan Kemiskinan"
    ]
    if not nilai_nduga_star.empty:
        nilai = nilai_nduga_star.values[0]
        mask_nduga = (final_df["Kabupaten/Kota"] == "Nduga") & (final_df["Tahun"] == tahun)
        if mask_nduga.any():
            if pd.isna(final_df.loc[mask_nduga, "Indeks Keparahan Kemiskinan"]).values[0]:
                final_df.loc[mask_nduga, "Indeks Keparahan Kemiskinan"] = nilai
        else:
            final_df = pd.concat([
                final_df,
                pd.DataFrame({
                    "Kabupaten/Kota": ["Nduga"],
                    "Tahun": [tahun],
                    "Indeks Keparahan Kemiskinan": [nilai]
                })
            ], ignore_index=True)

# 🛠 Tangani kasus khusus: Kota Sawah Lunto 2020 kosong → isi dari 2019
mask_2020 = (final_df["Kabupaten/Kota"] == "Kota Sawah Lunto") & (final_df["Tahun"] == 2020)
mask_2019 = (final_df["Kabupaten/Kota"] == "Kota Sawah Lunto") & (final_df["Tahun"] == 2019)

if mask_2020.any() and pd.isna(final_df.loc[mask_2020, "Indeks Keparahan Kemiskinan"]).values[0]:
    nilai_2019 = final_df.loc[mask_2019, "Indeks Keparahan Kemiskinan"].values[0]
    final_df.loc[mask_2020, "Indeks Keparahan Kemiskinan"] = nilai_2019

# Hapus entri tidak valid
final_df = final_df[~final_df["Kabupaten/Kota"].isin(["Nduga *", "KEP. BANGKA BELITUNG"])]

# Hilangkan duplikat
final_df = final_df.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Indeks Keparahan Kemiskinan"],
    ascending=[True, True, True]
)
final_df = final_df.drop_duplicates(
    subset=["Kabupaten/Kota", "Tahun"],
    keep="first"
).reset_index(drop=True)

# Simpan hasil
final_df.to_csv(output_path / "p2_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", final_df["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", final_df.shape[0])

📄 Membaca: Indeks Keparahan Kemiskinan (P2) Menurut Kabupaten_Kota, 2016-2018.xlsx
📄 Membaca: Indeks Keparahan Kemiskinan (P2) Menurut Kabupaten_Kota, 2019-2021.xlsx
📄 Membaca: Indeks Keparahan Kemiskinan (P2) Menurut Kabupaten_Kota, 2022-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [10]:
# Baca file dengan separator titik koma
file_path = data_folder / "Indeks Ketahanan Pangan.csv"
print(f"📄 Membaca: {file_path.name}")
df = pd.read_csv(file_path, sep=';', encoding='utf-8')

# Ambil kolom yang dibutuhkan
df_filtered = df[["Nama Provinsi", "Nama Kabupaten", "Tahun", "Kelompok IKP"]]

# Ganti nama kolom
df_filtered = df_filtered.rename(columns={
    "Nama Provinsi": "Provinsi",
    "Nama Kabupaten": "Kabupaten/Kota"
})

# Bersihkan nama kabupaten/kota
df_filtered["Kabupaten/Kota"] = (
    df_filtered["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Simpan hasil
df_filtered.to_csv(output_path / "ikp_long.csv", index=False)

# Ringkasan
print("✅ Jumlah kabupaten/kota unik:", df_filtered["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_filtered.shape[0])
df_filtered.head()

📄 Membaca: Indeks Ketahanan Pangan.csv
✅ Jumlah kabupaten/kota unik: 514
✅ Jumlah baris: 3084


,Provinsi,Kabupaten/Kota,Tahun,Kelompok IKP
0,Aceh,Simeulue,2018,5
1,Aceh,Aceh Singkil,2018,5
2,Aceh,Aceh Selatan,2018,5
3,Aceh,Aceh Tenggara,2018,2
4,Aceh,Aceh Timur,2018,4


In [11]:
# Daftar file yang akan dibaca
file_names = [
    "Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2016-2018.xlsx",
    "Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2019-2021.xlsx",
    "Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2022-2024.xlsx"
]

all_dfs = []

# Proses setiap file
for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_path.name}")
    df = pd.read_excel(file_path, na_values=["-"])

    df_long = df.melt(
        id_vars="Nama Wilayah",
        var_name="Tahun",
        value_name="Jumlah Penduduk Miskin"
    )

    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)
    all_dfs.append(df_long)

# Gabungkan semua data
final_df = pd.concat(all_dfs, ignore_index=True)

# Filter tahun ≥ 2018
final_df = final_df[final_df["Tahun"] >= 2018].reset_index(drop=True)

# Hapus baris provinsi (nama kapital semua)
final_df = final_df[~final_df["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Bersihkan nama wilayah
final_df["Kabupaten/Kota"] = (
    final_df["Nama Wilayah"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Hapus kolom lama
final_df = final_df.drop(columns="Nama Wilayah")

# Pindahkan nilai dari 'Nduga *' ke 'Nduga' jika kosong
for tahun in final_df["Tahun"].unique():
    nilai_nduga_star = final_df.loc[
        (final_df["Kabupaten/Kota"] == "Nduga *") & (final_df["Tahun"] == tahun),
        "Jumlah Penduduk Miskin"
    ].values

    if len(nilai_nduga_star) == 0:
        continue

    nilai = nilai_nduga_star[0]

    mask_nduga = (final_df["Kabupaten/Kota"] == "Nduga") & (final_df["Tahun"] == tahun)
    if mask_nduga.sum() > 0 and pd.isna(final_df.loc[mask_nduga, "Jumlah Penduduk Miskin"]).values[0]:
        final_df.loc[mask_nduga, "Jumlah Penduduk Miskin"] = nilai

# Hapus entri tidak valid
final_df = final_df[~final_df["Kabupaten/Kota"].isin(["Nduga *", "KEP. BANGKA BELITUNG"])]

# Prioritaskan data non-NaN jika duplikat
final_df = final_df.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Jumlah Penduduk Miskin"],
    ascending=[True, True, True]
).drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="first").reset_index(drop=True)

# Simpan ke CSV
final_df.to_csv(output_path / "jumlah_penduduk_miskin_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", final_df["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", final_df.shape[0])
final_df.head()

📄 Membaca: Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2016-2018.xlsx
📄 Membaca: Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2019-2021.xlsx
📄 Membaca: Jumlah Penduduk Miskin (Ribu Jiwa) Menurut Kabupaten_Kota , 2022-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


,Tahun,Jumlah Penduduk Miskin,Kabupaten/Kota
0,2018,39.56,Aceh Barat
1,2019,39.29,Aceh Barat
2,2020,39.06,Aceh Barat
3,2021,39.83,Aceh Barat
4,2022,38.46,Aceh Barat


In [12]:
file_path = data_folder / "Persentase Penduduk Miskin (P0) Menurut Kabupaten_Kota, 2018-2024.xlsx"

# Baca file
print(f"📄 Membaca: {file_path.name}")
df = pd.read_excel(file_path, na_values=["-"])

# Rename kolom pertama
df = df.rename(columns={df.columns[0]: "Nama Wilayah"})

# Wide ke long
df_long = df.melt(
    id_vars="Nama Wilayah", 
    var_name="Tahun", 
    value_name="Persentase Penduduk Miskin"
)

# Ekstrak angka tahun
df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

# Filter tahun 2018 ke atas
df_long = df_long[df_long["Tahun"] >= 2018].reset_index(drop=True)

# Hapus baris huruf kapital (provinsi)
df_long = df_long[~df_long["Nama Wilayah"].str.fullmatch(r"[A-Z\s]+")]

# Ubah nama kolom
df_long = df_long.rename(columns={"Nama Wilayah": "Kabupaten/Kota"})

# Bersihkan nama kabupaten/kota
df_long["Kabupaten/Kota"] = (
    df_long["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Pindahkan nilai dari 'Nduga *' ke 'Nduga' jika 'Nduga' kosong di tahun tersebut
for tahun in df_long["Tahun"].unique():
    nilai_nduga_star = df_long.loc[
        (df_long["Kabupaten/Kota"] == "Nduga *") & (df_long["Tahun"] == tahun),
        "Persentase Penduduk Miskin"
    ].values

    if len(nilai_nduga_star) > 0:
        nilai = nilai_nduga_star[0]
        mask_nduga = (df_long["Kabupaten/Kota"] == "Nduga") & (df_long["Tahun"] == tahun)
        if mask_nduga.any() and pd.isna(df_long.loc[mask_nduga, "Persentase Penduduk Miskin"]).values[0]:
            df_long.loc[mask_nduga, "Persentase Penduduk Miskin"] = nilai

# Hapus baris 'Nduga *' dan 'KEP. BANGKA BELITUNG'
df_long = df_long[~df_long["Kabupaten/Kota"].isin(["Nduga *", "KEP. BANGKA BELITUNG"])]

# Urutkan dan hapus duplikat (jika ada)
df_long = df_long.sort_values(
    by=["Kabupaten/Kota", "Tahun", "Persentase Penduduk Miskin"],
    ascending=[True, True, True]
)
df_long = df_long.drop_duplicates(
    subset=["Kabupaten/Kota", "Tahun"],
    keep="first"
).reset_index(drop=True)

# Simpan hasil akhir
df_long.to_csv(output_path / "persentase_penduduk_miskin_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_long["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_long.shape[0])

📄 Membaca: Persentase Penduduk Miskin (P0) Menurut Kabupaten_Kota, 2018-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [13]:
file_path = data_folder / "Prevalensi Ketidakcukupan Konsumsi Pangan (Persen) Per Kabupaten_kota, 2018-2024 (1).xlsx"

# Baca file
print(f"📄 Membaca: {file_path.name}")
df = pd.read_excel(file_path, na_values=["-"])

# Rename kolom pertama agar seragam
df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})

# Wide ke long
df_long = df.melt(
    id_vars="Kabupaten/Kota", 
    var_name="Tahun", 
    value_name="Prevalensi Konsumsi Pangan Tidak Cukup"
)

# Ekstrak tahun dari nama kolom
df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})").astype(int)

# Bersihkan nama kabupaten/kota
df_long["Kabupaten/Kota"] = (
    df_long["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Pindahkan nilai dari 'Nduga *' ke 'Nduga' jika kosong
for tahun in df_long["Tahun"].unique():
    nilai_nduga_star = df_long.loc[
        (df_long["Kabupaten/Kota"] == "Nduga *") & (df_long["Tahun"] == tahun),
        "Prevalensi Konsumsi Pangan Tidak Cukup"
    ].values

    if len(nilai_nduga_star) == 0:
        continue

    nilai = nilai_nduga_star[0]

    mask_nduga = (df_long["Kabupaten/Kota"] == "Nduga") & (df_long["Tahun"] == tahun)
    if mask_nduga.sum() > 0 and pd.isna(df_long.loc[mask_nduga, "Prevalensi Konsumsi Pangan Tidak Cukup"]).values[0]:
        df_long.loc[mask_nduga, "Prevalensi Konsumsi Pangan Tidak Cukup"] = nilai

# Hapus baris tidak valid
df_long = df_long[~df_long["Kabupaten/Kota"].isin(["Nduga *", "KEP. BANGKA BELITUNG"])]

# Simpan hasil
df_long.to_csv(output_path / "prevalensi_konsumsi_pangan_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_long["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_long.shape[0])

📄 Membaca: Prevalensi Ketidakcukupan Konsumsi Pangan (Persen) Per Kabupaten_kota, 2018-2024 (1).xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [14]:
file_path = data_folder / "Rata-rata Lama Sekolah (RLS) menurut Jenis Kelamin, 2018-2024.xlsx"
print(f"📄 Membaca: {file_path.name}")
df = pd.read_excel(file_path, na_values=["-"])

# Rename kolom nama daerah
df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})

# Bersihkan nama
df["Kabupaten/Kota"] = (
    df["Kabupaten/Kota"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Hapus baris yang merupakan nama provinsi (huruf kapital semua)
df = df[~df["Kabupaten/Kota"].str.fullmatch(r"[A-Z\s\/\(\)\.\-]+")]

# Pisahkan berdasarkan gender
df_lk = df.filter(regex="Kabupaten/Kota|Laki-laki")
df_lk = df_lk.melt(
    id_vars="Kabupaten/Kota", 
    var_name="Tahun", 
    value_name="Rata-rata Lama Sekolah_Laki-laki"
)
df_lk["Tahun"] = df_lk["Tahun"].str.extract(r"(\d{4})").astype(int)

df_pr = df.filter(regex="Kabupaten/Kota|Perempuan")
df_pr = df_pr.melt(
    id_vars="Kabupaten/Kota", 
    var_name="Tahun", 
    value_name="Rata-rata Lama Sekolah_Perempuan"
)
df_pr["Tahun"] = df_pr["Tahun"].str.extract(r"(\d{4})").astype(int)

# Gabung dua gender
df_final = pd.merge(df_lk, df_pr, on=["Kabupaten/Kota", "Tahun"], how="outer")

# Gabungkan duplikat → ambil nilai non-NaN pertama
df_final = df_final.groupby(["Kabupaten/Kota", "Tahun"], as_index=False).agg({
    "Rata-rata Lama Sekolah_Laki-laki": "first",
    "Rata-rata Lama Sekolah_Perempuan": "first"
})

# Urutkan dan simpan
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun"])
df_final.to_csv(output_path / "rata_rata_lama_sekolah_long.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Lama Sekolah (RLS) menurut Jenis Kelamin, 2018-2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [15]:
# Daftar file
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Makanan Lainnya Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Makanan Lainnya Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Makanan Lainnya Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])

    # Pastikan kolom nama daerah konsisten
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Wide ke long
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Bahan Makanan Lainnya"
    )

    # Ekstrak tahun, drop jika tidak valid
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")[0]
    df_long = df_long[df_long["Tahun"].notna()]
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan semua data
df_final = pd.concat(all_dfs, ignore_index=True)

# Bersihkan data
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Bahan Makanan Lainnya"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan hasil
df_final.to_csv(output_path / "pengeluaran_bahan_makanan_lainnya.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])


📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Makanan Lainnya Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Makanan Lainnya Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Makanan Lainnya Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [16]:
# File bahan minuman
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Minuman Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Minuman Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Minuman Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])

    # Pastikan kolom nama daerah konsisten
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Wide ke long
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Bahan Minuman"
    )

    # Ekstrak tahun (hindari error NaN ke int)
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long.dropna(subset=["Tahun"])
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan semua data
df_final = pd.concat(all_dfs, ignore_index=True)

# Bersihkan dan prioritaskan data non-NaN
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Bahan Minuman"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan hasil
df_final.to_csv(output_path / "pengeluaran_bahan_minuman.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Minuman Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Minuman Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Bahan Minuman Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [17]:
# Daftar file (gunakan tanda hubung - bukan en dash –)
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Buah-Buahan Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Buah-Buahan Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Buah-Buahan Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])

    # Pastikan nama kolom pertama adalah 'Kabupaten/Kota'
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Wide ke long
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Buah-Buahan"
    )

    # Ekstrak tahun dari nama kolom
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long.dropna(subset=["Tahun"])
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan dan bersihkan
df_final = pd.concat(all_dfs, ignore_index=True)
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Buah-Buahan"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan ke CSV
df_final.to_csv(output_path / "pengeluaran_buah_buahan.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Buah-Buahan Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Buah-Buahan Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Buah-Buahan Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [18]:
# Daftar nama file
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Daging Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Daging Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Daging Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")

    df = pd.read_excel(file_path, na_values=["-"])

    # Pastikan nama kolom pertama adalah 'Kabupaten/Kota'
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Wide ke long
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Daging"
    )

    # Ekstrak tahun dengan hati-hati karena bisa ada NaN
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long.dropna(subset=["Tahun"])
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan semua data
df_final = pd.concat(all_dfs, ignore_index=True)

# Imputasi khusus untuk Buton Selatan 2018 jika kosong → isi dari 2019
mask_2018 = (df_final["Kabupaten/Kota"] == "Buton Selatan") & (df_final["Tahun"] == 2018)
mask_2019 = (df_final["Kabupaten/Kota"] == "Buton Selatan") & (df_final["Tahun"] == 2019)

if mask_2018.any() and df_final.loc[mask_2018, "Pengeluaran Perkapita Daging"].isna().any():
    nilai_2019 = df_final.loc[mask_2019, "Pengeluaran Perkapita Daging"].values
    if len(nilai_2019) > 0:
        df_final.loc[mask_2018, "Pengeluaran Perkapita Daging"] = nilai_2019[0]
        print("✅ Nilai 2018 untuk Buton Selatan diisi dengan nilai dari 2019.")
    else:
        print("⚠️ Nilai 2019 untuk Buton Selatan tidak ditemukan.")

# Prioritaskan nilai non-NaN jika ada duplikat
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Daging"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan hasil
df_final.to_csv(output_path / "pengeluaran_daging.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Daging Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Daging Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Daging Per Kabupaten_kota, 2024.xlsx
✅ Nilai 2018 untuk Buton Selatan diisi dengan nilai dari 2019.
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [19]:
# Daftar nama file (pastikan sama persis dengan file di folder)
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Kacang-Kacangan Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Kacang-Kacangan Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Kacang-Kacangan Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")

    df = pd.read_excel(file_path, na_values=["-"])

    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Kacang-Kacangan"
    )

    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long[df_long["Tahun"].notna()]
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

df_final = pd.concat(all_dfs, ignore_index=True)

# 🔁 Imputasi Puncak 2018 dan 2020 dengan nilai 2019
nilai_2019 = df_final.loc[
    (df_final["Kabupaten/Kota"] == "Puncak") & (df_final["Tahun"] == 2019),
    "Pengeluaran Perkapita Kacang-Kacangan"
]

if not nilai_2019.empty:
    nilai = nilai_2019.values[0]
    for tahun in [2018, 2020]:
        mask = (df_final["Kabupaten/Kota"] == "Puncak") & (df_final["Tahun"] == tahun)
        if mask.any():
            if pd.isna(df_final.loc[mask, "Pengeluaran Perkapita Kacang-Kacangan"]).values[0]:
                df_final.loc[mask, "Pengeluaran Perkapita Kacang-Kacangan"] = nilai
        else:
            df_final = pd.concat([
                df_final,
                pd.DataFrame({
                    "Kabupaten/Kota": ["Puncak"],
                    "Tahun": [tahun],
                    "Pengeluaran Perkapita Kacang-Kacangan": [nilai]
                })
            ], ignore_index=True)

# Hapus duplikat dan urutkan
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Kacang-Kacangan"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan ke CSV
df_final.to_csv(output_path / "pengeluaran_kacang_kacangan.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Kacang-Kacangan Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Kacang-Kacangan Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Kacang-Kacangan Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [20]:
# Daftar file
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Minyak dan Kelapa Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Minyak dan Kelapa Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Minyak dan Kelapa Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")

    # Baca data
    df = pd.read_excel(file_path, na_values=["-"])

    # Standarisasi nama kolom
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Wide ke long
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Bahan Minyak dan Kelapa"
    )

    # Ekstrak angka tahun
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long[df_long["Tahun"].notna()]
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan semua data
df_final = pd.concat(all_dfs, ignore_index=True)

# Bersihkan data duplikat
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Bahan Minyak dan Kelapa"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan hasil
df_final.to_csv(output_path / "pengeluaran_minyak_kelapa.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Minyak dan Kelapa Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Minyak dan Kelapa Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Minyak dan Kelapa Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [21]:
# Daftar file
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Padi-Padian Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Padi-Padian Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Padi-Padian Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")

    # Baca dan bersihkan data
    df = pd.read_excel(file_path, na_values=["-"])
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Wide ke long
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Padi-Padian"
    )

    # Ekstrak tahun dan ubah jadi int, abaikan NaN
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long.dropna(subset=["Tahun"])
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan semua data
df_final = pd.concat(all_dfs, ignore_index=True)

# Prioritaskan nilai non-NaN jika ada duplikat
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Padi-Padian"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan hasil
df_final.to_csv(output_path / "pengeluaran_padi_padian.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Padi-Padian Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Padi-Padian Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Padi-Padian Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [22]:
# Daftar nama file
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Rokok dan Tembakau Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Rokok dan Tembakau Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Rokok dan Tembakau Per Kabupaten_kota, 2024.xlsx"
]

all_dfs = []

# Proses tiap file
for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Rokok dan Tembakau"
    )
    df_long["Tahun"] = df_long["Tahun"].astype(str).str.extract(r"(\d{4})")
    df_long = df_long[df_long["Tahun"].notna()]
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan semua
df_final = pd.concat(all_dfs, ignore_index=True)

# Prioritaskan nilai bukan NaN jika ada duplikat
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Rokok dan Tembakau"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan
df_final.to_csv(output_path / "pengeluaran_rokok_tembakau.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Rokok dan Tembakau Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Rokok dan Tembakau Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Rokok dan Tembakau Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [23]:
# Daftar file
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Telur dan Susu Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Telur dan Susu Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Telur dan Susu Per Kabupaten_kota, 2024.xlsx"
]

# Gabung semua file
all_dfs = []

for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    # Baca file
    df = pd.read_excel(file_path, na_values=["-"])

    # Pastikan kolom pertama adalah nama wilayah
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Reshape
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Telur dan Susu"
    )

    # Ekstrak tahun dari kolom
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long[df_long["Tahun"].notna()]
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan
df_final = pd.concat(all_dfs, ignore_index=True)

# Hapus duplikat, prioritaskan yang valid
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Telur dan Susu"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan
df_final.to_csv(output_path / "pengeluaran_telur_susu.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Telur dan Susu Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Telur dan Susu Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Telur dan Susu Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [24]:
# Daftar file Excel
file_names = [
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Umbi-Umbian Per Kabupaten_kota, 2018-2020.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Umbi-Umbian Per Kabupaten_kota, 2021-2023.xlsx",
    "Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Umbi-Umbian Per Kabupaten_kota, 2024.xlsx"
]

# List penampung
all_dfs = []

# Proses masing-masing file
for file_name in file_names:
    file_path = data_folder / file_name
    print(f"📄 Membaca: {file_name}")
    
    df = pd.read_excel(file_path, na_values=["-"])
    
    # Pastikan kolom pertama konsisten
    df = df.rename(columns={df.columns[0]: "Kabupaten/Kota"})
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].str.strip().str.replace(r"\s+", " ", regex=True)

    # Ubah wide ke long
    df_long = df.melt(
        id_vars="Kabupaten/Kota",
        var_name="Tahun",
        value_name="Pengeluaran Perkapita Umbi-Umbian"
    )
    
    # Ekstrak tahun dari nama kolom
    df_long["Tahun"] = df_long["Tahun"].str.extract(r"(\d{4})")
    df_long = df_long[df_long["Tahun"].notna()]  # Drop jika gagal ekstrak tahun
    df_long["Tahun"] = df_long["Tahun"].astype(int)

    all_dfs.append(df_long)

# Gabungkan semua file
df_final = pd.concat(all_dfs, ignore_index=True)

# Urut dan bersihkan duplikat
df_final = df_final.sort_values(["Kabupaten/Kota", "Tahun", "Pengeluaran Perkapita Umbi-Umbian"])
df_final = df_final.drop_duplicates(subset=["Kabupaten/Kota", "Tahun"], keep="last").reset_index(drop=True)

# Simpan hasil
df_final.to_csv(output_path / "pengeluaran_umbi_umbian.csv", index=False)

# Ringkasan
print("✅ Jumlah Kab/Kota unik:", df_final["Kabupaten/Kota"].nunique())
print("✅ Jumlah baris:", df_final.shape[0])

📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Umbi-Umbian Per Kabupaten_kota, 2018-2020.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Umbi-Umbian Per Kabupaten_kota, 2021-2023.xlsx
📄 Membaca: Rata-rata Pengeluaran Perkapita Seminggu  Menurut Kelompok Umbi-Umbian Per Kabupaten_kota, 2024.xlsx
✅ Jumlah Kab/Kota unik: 514
✅ Jumlah baris: 3598


In [25]:
# Path folder Data Clean
clean_path = Path("../data/Data Clean")

# Tampilkan semua file di folder tersebut
file_list = list(clean_path.glob("*.csv"))  # * artinya semua file dan folder

# Cetak nama file
print("📁 Daftar file di Data Clean:")
for file in file_list:
    print("-", file.name)

📁 Daftar file di Data Clean:
- ahh_long.csv
- fsva_clean.csv
- garis_kemiskinan_long.csv
- ikp_long.csv
- ipm_long.csv
- jumlah_penduduk_miskin_long.csv
- p1_long.csv
- p2_long.csv
- pengeluaran_bahan_makanan_lainnya.csv
- pengeluaran_bahan_minuman.csv
- pengeluaran_buah_buahan.csv
- pengeluaran_daging.csv
- pengeluaran_kacang_kacangan.csv
- pengeluaran_minyak_kelapa.csv
- pengeluaran_padi_padian.csv
- pengeluaran_rokok_tembakau.csv
- pengeluaran_telur_susu.csv
- pengeluaran_umbi_umbian.csv
- persentase_penduduk_miskin_long.csv
- prevalensi_konsumsi_pangan_long.csv
- rata_rata_lama_sekolah_long.csv


In [26]:
# Inisialisasi dictionary untuk menyimpan DataFrame
all_xdfs = {}

# Loop untuk membaca file .csv kecuali 'ikp_long.csv'
for file in clean_path.glob("*.csv"):
    if file.name != "ikp_long.csv":
        print(f"📄 Membaca: {file.name}")
        df = pd.read_csv(file)
        all_xdfs[file.stem] = df  # file.stem = nama file tanpa ekstensi

📄 Membaca: ahh_long.csv
📄 Membaca: fsva_clean.csv
📄 Membaca: garis_kemiskinan_long.csv
📄 Membaca: ipm_long.csv
📄 Membaca: jumlah_penduduk_miskin_long.csv
📄 Membaca: p1_long.csv
📄 Membaca: p2_long.csv
📄 Membaca: pengeluaran_bahan_makanan_lainnya.csv
📄 Membaca: pengeluaran_bahan_minuman.csv
📄 Membaca: pengeluaran_buah_buahan.csv
📄 Membaca: pengeluaran_daging.csv
📄 Membaca: pengeluaran_kacang_kacangan.csv
📄 Membaca: pengeluaran_minyak_kelapa.csv
📄 Membaca: pengeluaran_padi_padian.csv
📄 Membaca: pengeluaran_rokok_tembakau.csv
📄 Membaca: pengeluaran_telur_susu.csv
📄 Membaca: pengeluaran_umbi_umbian.csv
📄 Membaca: persentase_penduduk_miskin_long.csv
📄 Membaca: prevalensi_konsumsi_pangan_long.csv
📄 Membaca: rata_rata_lama_sekolah_long.csv


In [27]:
# Ambil set referensi dari ikp_long
ikp_df = pd.read_csv(clean_path / "ikp_long.csv")
ref_set = set(ikp_df["Kabupaten/Kota"].dropna().unique())
print(f"✅ Referensi: ikp_long.csv ({len(ref_set)} kabupaten/kota)\n")

# Cek setiap file di xfile_paths
for name, df in all_xdfs.items():
    if "Kabupaten/Kota" in df.columns:
        this_set = set(df["Kabupaten/Kota"].dropna().unique())
        if this_set != ref_set:
            diff1 = ref_set - this_set
            diff2 = this_set - ref_set
            print(f"❌ Perbedaan ditemukan di '{name}':")
            if diff1:
                print(f"   - Tidak ada di {name}: {sorted(diff1)}")
            if diff2:
                print(f"   - Tambahan di {name}: {sorted(diff2)}")
            print()
        else:
            print(f"✅ '{name}' cocok dengan referensi.")
    else:
        print(f"⚠️ Kolom 'Kabupaten/Kota' tidak ditemukan di '{name}'")

✅ Referensi: ikp_long.csv (514 kabupaten/kota)

❌ Perbedaan ditemukan di 'ahh_long':
   - Tidak ada di ahh_long: ['Banyuasin', 'Biak Namfor', 'Dharmas Raya', 'Fak-Fak', 'Kepulauan Seribu', 'Kepulauan Siau Tagulandang Biaro', 'Kepulauan Tanimbar', 'Kota Baru', 'Kota Bau-Bau', 'Kota Kotamobago', 'Kota Padang Sidimpuan', 'Kota Pare-Pare', 'Kuantan Sengingi', 'Kutai Kertanegara', 'Limapuluh Koto', 'Mamuju Utara', 'Melawai', 'Selayar', 'Toba Samosir', 'Tulang Bawang']
   - Tambahan di ahh_long: ['Banyu Asin', 'Biak Numfor', 'Dharmasraya', 'Fakfak', 'Kep. Seribu', 'Kepulauan Selayar', 'Kota Baubau', 'Kota Kotamobagu', 'Kota Padangsidimpuan', 'Kota Parepare', 'Kotabaru', 'Kuantan Singingi', 'Kutai Kartanegara', 'Lima Puluh Kota', 'Maluku Tenggara Barat / Kepulauan Tanimbar', 'Mamuju Utara / Pasangkayu', 'Melawi', 'Siau Tagulandang Biaro', 'Toba Samosir / Toba', 'Tulangbawang']

❌ Perbedaan ditemukan di 'fsva_clean':
   - Tidak ada di fsva_clean: ['Batang Hari', 'Biak Namfor', 'Dharmas Raya', 

In [28]:
# 1. Mapping koreksi nama kabupaten/kota agar sesuai dengan ikp_long
correction_map = {
    "Banyu Asin": "Banyuasin",
    "Biak Numfor": "Biak Namfor",
    "Dharmasraya": "Dharmas Raya",
    "Fakfak": "Fak-Fak",
    "Kep. Seribu": "Kepulauan Seribu",
    "Kepulauan Selayar": "Selayar",
    "Kota Baubau": "Kota Bau-Bau",
    "Kota Kotamobagu": "Kota Kotamobago",
    "Kota Padangsidimpuan": "Kota Padang Sidimpuan",
    "Kota Parepare": "Kota Pare-Pare",
    "Kotabaru": "Kota Baru",
    "Kuantan Singingi": "Kuantan Sengingi",
    "Kutai Kartanegara": "Kutai Kertanegara",
    "Lima Puluh Kota": "Limapuluh Koto",
    "Maluku Tenggara Barat": "Kepulauan Tanimbar",
    "Maluku Tenggara Barat / Kepulauan Tanimbar": "Kepulauan Tanimbar",
    "Mamuju Utara / Pasangkayu": "Mamuju Utara",
    "Pasangkayu": "Mamuju Utara",
    "Pontianak": "Mempawah",
    "Melawi": "Melawai",
    "Siau Tagulandang Biaro": "Kepulauan Siau Tagulandang Biaro",
    "Toba Samosir / Toba": "Toba Samosir",
    "Tulangbawang": "Tulang Bawang",
    "Karang Asem": "Karangasem",
    "Kota Makassar": "Kota Makasar",
    "Mahakam Hulu": "Mahakam Ulu",
    "Pangkajene Dan Kepulauan": "Pangkajene dan Kepulauan",
    
    # Tambahan dari hasil fsva_clean
    "Batanghari": "Batang Hari",
    "Gunungkidul": "Gunung Kidul",
    "Jakarta Barat": "Kota Jakarta Barat",
    "Jakarta Pusat": "Kota Jakarta Pusat",
    "Jakarta Selatan": "Kota Jakarta Selatan",
    "Jakarta Timur": "Kota Jakarta Timur",
    "Jakarta Utara": "Kota Jakarta Utara",
    "Kep. Siau Tagulandang Biaro": "Kepulauan Siau Tagulandang Biaro",
    "Kota Banjarbaru": "Kota Banjar Baru",
    "Kota Bau Bau": "Kota Bau-Bau",
    "Kota Palangkaraya": "Kota Palangka Raya",
    "Kota Pangkalpinang": "Kota Pangkal Pinang",
    "Kota Pare Pare": "Kota Pare-Pare",
    "Kota Pematangsiantar": "Kota Pematang Siantar",
    "Kota Sawahlunto": "Kota Sawah Lunto",
    "Kota Tanjungpinang": "Kota Tanjung Pinang",
    "Labuhanbatu": "Labuhan Batu",
    "Labuhanbatu Selatan": "Labuhan Batu Selatan",
    "Labuhanbatu Utara": "Labuhan Batu Utara",
    "Pahuwato": "Pohuwato",
    "Tolitoli": "Toli-Toli"
}

# 2. Terapkan mapping ke semua file di all_xdfs
for name, df in all_xdfs.items():
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].replace(correction_map)
    all_xdfs[name] = df  # Perbarui dict-nya (opsional jika inplace)

print("✅ Mapping nama kabupaten/kota telah diterapkan ke semua file fitur.")

✅ Mapping nama kabupaten/kota telah diterapkan ke semua file fitur.


In [29]:
print(f"✅ Referensi: ikp_long.csv ({len(ref_set)} kabupaten/kota)\n")

# Cek setiap file di xfile_paths
for name, df in all_xdfs.items():
    if "Kabupaten/Kota" in df.columns:
        this_set = set(df["Kabupaten/Kota"].dropna().unique())
        if this_set != ref_set:
            diff1 = ref_set - this_set
            diff2 = this_set - ref_set
            print(f"❌ Perbedaan ditemukan di '{name}':")
            if diff1:
                print(f"   - Tidak ada di {name}: {sorted(diff1)}")
            if diff2:
                print(f"   - Tambahan di {name}: {sorted(diff2)}")
            print()
        else:
            print(f"✅ '{name}' cocok dengan referensi.")
    else:
        print(f"⚠️ Kolom 'Kabupaten/Kota' tidak ditemukan di '{name}'")

✅ Referensi: ikp_long.csv (514 kabupaten/kota)

✅ 'ahh_long' cocok dengan referensi.
✅ 'fsva_clean' cocok dengan referensi.
✅ 'garis_kemiskinan_long' cocok dengan referensi.
✅ 'ipm_long' cocok dengan referensi.
✅ 'jumlah_penduduk_miskin_long' cocok dengan referensi.
✅ 'p1_long' cocok dengan referensi.
✅ 'p2_long' cocok dengan referensi.
✅ 'pengeluaran_bahan_makanan_lainnya' cocok dengan referensi.
✅ 'pengeluaran_bahan_minuman' cocok dengan referensi.
✅ 'pengeluaran_buah_buahan' cocok dengan referensi.
✅ 'pengeluaran_daging' cocok dengan referensi.
✅ 'pengeluaran_kacang_kacangan' cocok dengan referensi.
✅ 'pengeluaran_minyak_kelapa' cocok dengan referensi.
✅ 'pengeluaran_padi_padian' cocok dengan referensi.
✅ 'pengeluaran_rokok_tembakau' cocok dengan referensi.
✅ 'pengeluaran_telur_susu' cocok dengan referensi.
✅ 'pengeluaran_umbi_umbian' cocok dengan referensi.
✅ 'persentase_penduduk_miskin_long' cocok dengan referensi.
✅ 'prevalensi_konsumsi_pangan_long' cocok dengan referensi.
✅ 'rat

In [30]:
# Mapping final nama kabupaten/kota
final_name_map = {
    "Fak-Fak": "Fakfak",
    "Selayar": "Kepulauan Selayar",
    "Kota Bau-Bau": "Kota Baubau",
    "Kota Kotamobago": "Kotamobagu",
    "Kota Padang Sidimpuan": "Kota Padangsidimpuan",
    "Kota Pare-Pare": "Kota Parepare",
    "Kota Baru": "Kotabaru",
    "Kuantan Sengingi": "Kuantan Singingi",
    "Kutai Kertanegara": "Kutai Kartanegara",
    "Limapuluh Koto": "Lima Puluh Kota",
    "Mamuju Utara": "Pasangkayu",
    "Melawai": "Melawi",
    "Kota Makasar": "Kota Makassar",
    "Mahakam Ulu": "Mahakam Hulu",
}

# Terapkan ke ikp_df
ikp_df["Kabupaten/Kota"] = ikp_df["Kabupaten/Kota"].replace(final_name_map)

# Terapkan ke semua df dalam all_xdfs
for name, df in all_xdfs.items():
    df["Kabupaten/Kota"] = df["Kabupaten/Kota"].replace(final_name_map)
    all_xdfs[name] = df  # tidak wajib, karena replace() inplace untuk Series

print("✅ Semua DataFrame telah dimapping ulang menggunakan nama final.")

✅ Semua DataFrame telah dimapping ulang menggunakan nama final.


In [31]:
# Ambil set referensi dari ikp_long
ref_set = set(ikp_df["Kabupaten/Kota"].dropna().unique())
print(f"✅ Referensi: ikp_long.csv ({len(ref_set)} kabupaten/kota)\n")

# Cek setiap file di xfile_paths
for name, df in all_xdfs.items():
    if "Kabupaten/Kota" in df.columns:
        this_set = set(df["Kabupaten/Kota"].dropna().unique())
        if this_set != ref_set:
            diff1 = ref_set - this_set
            diff2 = this_set - ref_set
            print(f"❌ Perbedaan ditemukan di '{name}':")
            if diff1:
                print(f"   - Tidak ada di {name}: {sorted(diff1)}")
            if diff2:
                print(f"   - Tambahan di {name}: {sorted(diff2)}")
            print()
        else:
            print(f"✅ '{name}' cocok dengan referensi.")
    else:
        print(f"⚠️ Kolom 'Kabupaten/Kota' tidak ditemukan di '{name}'")

✅ Referensi: ikp_long.csv (514 kabupaten/kota)

✅ 'ahh_long' cocok dengan referensi.
✅ 'fsva_clean' cocok dengan referensi.
✅ 'garis_kemiskinan_long' cocok dengan referensi.
✅ 'ipm_long' cocok dengan referensi.
✅ 'jumlah_penduduk_miskin_long' cocok dengan referensi.
✅ 'p1_long' cocok dengan referensi.
✅ 'p2_long' cocok dengan referensi.
✅ 'pengeluaran_bahan_makanan_lainnya' cocok dengan referensi.
✅ 'pengeluaran_bahan_minuman' cocok dengan referensi.
✅ 'pengeluaran_buah_buahan' cocok dengan referensi.
✅ 'pengeluaran_daging' cocok dengan referensi.
✅ 'pengeluaran_kacang_kacangan' cocok dengan referensi.
✅ 'pengeluaran_minyak_kelapa' cocok dengan referensi.
✅ 'pengeluaran_padi_padian' cocok dengan referensi.
✅ 'pengeluaran_rokok_tembakau' cocok dengan referensi.
✅ 'pengeluaran_telur_susu' cocok dengan referensi.
✅ 'pengeluaran_umbi_umbian' cocok dengan referensi.
✅ 'persentase_penduduk_miskin_long' cocok dengan referensi.
✅ 'prevalensi_konsumsi_pangan_long' cocok dengan referensi.
✅ 'rat

In [32]:
print("📁 DataFrame yang masuk ke 'merged_df':")
for name in all_xdfs.keys():
    print("-", name)

📁 DataFrame yang masuk ke 'merged_df':
- ahh_long
- fsva_clean
- garis_kemiskinan_long
- ipm_long
- jumlah_penduduk_miskin_long
- p1_long
- p2_long
- pengeluaran_bahan_makanan_lainnya
- pengeluaran_bahan_minuman
- pengeluaran_buah_buahan
- pengeluaran_daging
- pengeluaran_kacang_kacangan
- pengeluaran_minyak_kelapa
- pengeluaran_padi_padian
- pengeluaran_rokok_tembakau
- pengeluaran_telur_susu
- pengeluaran_umbi_umbian
- persentase_penduduk_miskin_long
- prevalensi_konsumsi_pangan_long
- rata_rata_lama_sekolah_long


In [33]:
# Konversi dict ke list DataFrame yang siap digabung
dfs = list(all_xdfs.values())

# Lakukan join satu per satu berdasarkan Kabupaten/Kota dan Tahun
merged_df = reduce(
    lambda left, right: pd.merge(left, right, on=["Kabupaten/Kota", "Tahun"], how="outer"),
    dfs
)

# Tampilkan info
print("✅ Data gabungan selesai.")
print("🔢 Total kolom:", len(merged_df.columns))
print("🔢 Total baris:", len(merged_df))

merged_df

✅ Data gabungan selesai.
🔢 Total kolom: 32
🔢 Total baris: 3598


,Kabupaten/Kota,Tahun,Angka Harapan Hidup,Kelompok IKP,NCPR,Kemiskinan (%),Pengeluaran Pangan (%),Tanpa Listrik (%),Tanpa Air Bersih (%),Lama Sekolah Perempuan (tahun),...,Pengeluaran Perkapita Kacang-Kacangan,Pengeluaran Perkapita Bahan Minyak dan Kelapa,Pengeluaran Perkapita Padi-Padian,Pengeluaran Perkapita Rokok dan Tembakau,Pengeluaran Perkapita Telur dan Susu,Pengeluaran Perkapita Umbi-Umbian,Persentase Penduduk Miskin,Prevalensi Konsumsi Pangan Tidak Cukup,Rata-rata Lama Sekolah_Laki-laki,Rata-rata Lama Sekolah_Perempuan
0,Aceh Barat,2018,65.69,5,0.42,20.28,42.10,2.29,33.76,9.38,...,1574.21,3603.83,17478.50,25505.96,5793.05,1016.09,19.31,8.01,9.45,8.83
1,Aceh Barat,2019,65.89,5,0.68,19.31,40.58,1.45,29.88,9.32,...,1554.97,4105.63,17903.88,22338.77,5385.41,997.98,18.79,7.74,9.46,8.84
2,Aceh Barat,2020,65.95,6,0.71,18.79,20.21,0.48,20.25,8.92,...,2049.18,4292.58,17316.41,26227.26,6762.96,1377.97,18.34,6.40,9.71,9.15
3,Aceh Barat,2021,65.96,6,0.56,18.34,29.72,0.43,24.74,9.63,...,2187.72,4772.75,20506.28,30775.45,6965.66,1632.07,18.81,4.09,9.82,9.39
4,Aceh Barat,2022,66.13,6,0.56,18.81,38.49,0.00,21.76,10.22,...,2194.79,5623.14,16938.04,29389.57,6443.15,2018.24,17.93,6.55,10.15,9.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,Yalimo,2020,63.39,1,5.00,34.52,52.99,14.09,81.54,3.61,...,3786.20,5363.85,29316.95,23952.91,7934.47,39612.33,32.82,22.28,4.73,2.75
3594,Yalimo,2021,63.40,1,5.00,32.82,28.61,35.16,71.53,4.99,...,2949.47,8366.30,29876.77,17731.74,10774.34,68226.64,33.25,29.08,4.93,2.99
3595,Yalimo,2022,63.56,1,5.00,33.25,69.23,27.11,100.00,5.66,...,2499.19,7063.01,24917.04,15917.07,8517.53,72288.14,31.68,26.29,5.01,3.18
3596,Yalimo,2023,64.05,1,5.00,31.68,71.76,33.36,100.00,10.67,...,3175.15,9355.16,38772.50,32682.78,15634.79,49610.61,30.97,17.94,5.13,3.34


In [34]:
# Ambil nama kolom dari fsva (kecuali 'Kabupaten/Kota' dan 'Tahun')
fsva_cols = [col for col in all_xdfs["fsva_clean"].columns if col not in ["Kabupaten/Kota", "Tahun"]]

# Susun ulang kolom: non-fsva dulu, lalu fsva
ordered_cols = [col for col in merged_df.columns if col not in fsva_cols] + fsva_cols
merged_df = merged_df[ordered_cols]

# Cek ulang struktur kolom
print("✅ Kolom dari FSVA telah dipindah ke kanan.")

✅ Kolom dari FSVA telah dipindah ke kanan.


In [35]:
# Jumlah unique Kabupaten/Kota dan Tahun
n_kabkot = merged_df["Kabupaten/Kota"].nunique()
n_tahun = merged_df["Tahun"].nunique()

# Daftar nilai unik
list_kabkot = sorted(merged_df["Kabupaten/Kota"].dropna().unique())
list_tahun = sorted(merged_df["Tahun"].dropna().unique())

# Tampilkan hasil
print(f"✅ Jumlah kabupaten/kota unik: {n_kabkot}")
print(f"📅 Daftar kabupaten/kota: {list_kabkot}\n")

print(f"✅ Jumlah tahun unik: {n_tahun}")
print(f"📅 Daftar tahun: {list_tahun}")


✅ Jumlah kabupaten/kota unik: 514
📅 Daftar kabupaten/kota: ['Aceh Barat', 'Aceh Barat Daya', 'Aceh Besar', 'Aceh Jaya', 'Aceh Selatan', 'Aceh Singkil', 'Aceh Tamiang', 'Aceh Tengah', 'Aceh Tenggara', 'Aceh Timur', 'Aceh Utara', 'Agam', 'Alor', 'Asahan', 'Asmat', 'Badung', 'Balangan', 'Bandung', 'Bandung Barat', 'Banggai', 'Banggai Kepulauan', 'Banggai Laut', 'Bangka', 'Bangka Barat', 'Bangka Selatan', 'Bangka Tengah', 'Bangkalan', 'Bangli', 'Banjar', 'Banjarnegara', 'Bantaeng', 'Bantul', 'Banyuasin', 'Banyumas', 'Banyuwangi', 'Barito Kuala', 'Barito Selatan', 'Barito Timur', 'Barito Utara', 'Barru', 'Batang', 'Batang Hari', 'Batu Bara', 'Bekasi', 'Belitung', 'Belitung Timur', 'Belu', 'Bener Meriah', 'Bengkalis', 'Bengkayang', 'Bengkulu Selatan', 'Bengkulu Tengah', 'Bengkulu Utara', 'Berau', 'Biak Namfor', 'Bima', 'Bintan', 'Bireuen', 'Blitar', 'Blora', 'Boalemo', 'Bogor', 'Bojonegoro', 'Bolaang Mongondow', 'Bolaang Mongondow Selatan', 'Bolaang Mongondow Timur', 'Bolaang Mongondow Utara

In [36]:
for i, col in enumerate(merged_df.columns, start=1):
    print(f"{i:02d}. {col}")

01. Kabupaten/Kota
02. Tahun
03. Angka Harapan Hidup
04. Garis Kemiskinan
05. Indeks Pembangunan Manusia
06. Jumlah Penduduk Miskin
07. Indeks Kedalaman Kemiskinan
08. Indeks Keparahan Kemiskinan
09. Pengeluaran Perkapita Bahan Makanan Lainnya
10. Pengeluaran Perkapita Bahan Minuman
11. Pengeluaran Perkapita Buah-Buahan
12. Pengeluaran Perkapita Daging
13. Pengeluaran Perkapita Kacang-Kacangan
14. Pengeluaran Perkapita Bahan Minyak dan Kelapa
15. Pengeluaran Perkapita Padi-Padian
16. Pengeluaran Perkapita Rokok dan Tembakau
17. Pengeluaran Perkapita Telur dan Susu
18. Pengeluaran Perkapita Umbi-Umbian
19. Persentase Penduduk Miskin
20. Prevalensi Konsumsi Pangan Tidak Cukup
21. Rata-rata Lama Sekolah_Laki-laki
22. Rata-rata Lama Sekolah_Perempuan
23. Kelompok IKP
24. NCPR
25. Kemiskinan (%)
26. Pengeluaran Pangan (%)
27. Tanpa Listrik (%)
28. Tanpa Air Bersih (%)
29. Lama Sekolah Perempuan (tahun)
30. Rasio Tenaga Kesehatan
31. Angka Harapan Hidup (tahun)
32. Stunting (%)


In [37]:
# Drop dua kolom yang tidak diperlukan
merged_df = merged_df.drop(columns=[
    "Angka Harapan Hidup",
    "Lama Sekolah Perempuan (tahun)"
], errors="ignore")  # 'errors=ignore' agar tidak error jika kolom tidak ada

# Rename kolom dari FSVA
merged_df = merged_df.rename(columns={
    "Angka Harapan Hidup (tahun)": "Angka Harapan Hidup"
})

print("✅ Kolom dibersihkan dan diubah sesuai instruksi.")


✅ Kolom dibersihkan dan diubah sesuai instruksi.


In [38]:
for i, col in enumerate(merged_df.columns, start=1):
    print(f"{i:02d}. {col}")

01. Kabupaten/Kota
02. Tahun
03. Garis Kemiskinan
04. Indeks Pembangunan Manusia
05. Jumlah Penduduk Miskin
06. Indeks Kedalaman Kemiskinan
07. Indeks Keparahan Kemiskinan
08. Pengeluaran Perkapita Bahan Makanan Lainnya
09. Pengeluaran Perkapita Bahan Minuman
10. Pengeluaran Perkapita Buah-Buahan
11. Pengeluaran Perkapita Daging
12. Pengeluaran Perkapita Kacang-Kacangan
13. Pengeluaran Perkapita Bahan Minyak dan Kelapa
14. Pengeluaran Perkapita Padi-Padian
15. Pengeluaran Perkapita Rokok dan Tembakau
16. Pengeluaran Perkapita Telur dan Susu
17. Pengeluaran Perkapita Umbi-Umbian
18. Persentase Penduduk Miskin
19. Prevalensi Konsumsi Pangan Tidak Cukup
20. Rata-rata Lama Sekolah_Laki-laki
21. Rata-rata Lama Sekolah_Perempuan
22. Kelompok IKP
23. NCPR
24. Kemiskinan (%)
25. Pengeluaran Pangan (%)
26. Tanpa Listrik (%)
27. Tanpa Air Bersih (%)
28. Rasio Tenaga Kesehatan
29. Angka Harapan Hidup
30. Stunting (%)


In [39]:
# Pastikan kolom ada
if "Kelompok IKP" in merged_df.columns:
    # Ambil semua kolom kecuali Kelompok IKP
    cols = [col for col in merged_df.columns if col != "Kelompok IKP"]
    # Tambahkan Kelompok IKP di akhir
    merged_df = merged_df[cols + ["Kelompok IKP"]]
    print("✅ Kolom 'Kelompok IKP' telah dipindahkan ke paling kanan.")
else:
    print("❌ Kolom 'Kelompok IKP' tidak ditemukan.")

✅ Kolom 'Kelompok IKP' telah dipindahkan ke paling kanan.


In [40]:
merged_df

,Kabupaten/Kota,Tahun,Garis Kemiskinan,Indeks Pembangunan Manusia,Jumlah Penduduk Miskin,Indeks Kedalaman Kemiskinan,Indeks Keparahan Kemiskinan,Pengeluaran Perkapita Bahan Makanan Lainnya,Pengeluaran Perkapita Bahan Minuman,Pengeluaran Perkapita Buah-Buahan,...,Rata-rata Lama Sekolah_Perempuan,NCPR,Kemiskinan (%),Pengeluaran Pangan (%),Tanpa Listrik (%),Tanpa Air Bersih (%),Rasio Tenaga Kesehatan,Angka Harapan Hidup,Stunting (%),Kelompok IKP
0,Aceh Barat,2018,447089.0,70.47,39.56,3.91,1.13,1596.38,4203.72,5558.00,...,8.83,0.42,20.28,42.10,2.29,33.76,2.32,67.62,33.20,5
1,Aceh Barat,2019,471058.0,71.22,39.29,3.53,0.97,1617.34,3978.93,7009.00,...,8.84,0.68,19.31,40.58,1.45,29.88,2.75,67.72,32.63,5
2,Aceh Barat,2020,517264.0,71.38,39.06,2.74,0.68,1906.06,3908.43,8432.00,...,9.15,0.71,18.79,20.21,0.48,20.25,2.73,67.93,0.00,6
3,Aceh Barat,2021,533712.0,71.67,39.83,3.82,1.09,2064.32,3931.38,9377.00,...,9.39,0.56,18.34,29.72,0.43,24.74,2.38,67.98,0.00,6
4,Aceh Barat,2022,558638.0,72.34,38.46,2.22,0.45,2011.67,4036.28,11058.28,...,9.58,0.56,18.81,38.49,0.00,21.76,1.82,67.99,27.40,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,Yalimo,2020,347171.0,48.34,20.84,11.47,4.86,6453.37,6709.25,3660.00,...,2.75,5.00,34.52,52.99,14.09,81.54,30.75,65.34,8.65,1
3594,Yalimo,2021,378167.0,49.01,22.32,13.86,7.28,7375.07,10846.26,8033.00,...,2.99,5.00,32.82,28.61,35.16,71.53,30.85,65.42,0.00,1
3595,Yalimo,2022,395868.0,49.90,21.72,9.68,4.22,7625.46,9882.05,7903.15,...,3.18,5.00,33.25,69.23,27.11,100.00,16.94,65.49,44.10,1
3596,Yalimo,2023,434256.0,50.87,21.67,11.89,6.94,9309.60,19151.62,11369.67,...,3.34,5.00,31.68,71.76,33.36,100.00,0.12,65.72,19.60,1


In [41]:
merged_df["Kabupaten/Kota"] = merged_df["Kabupaten/Kota"].replace({
    "Kotamobagu": "Kota Kotamobagu"
})

In [42]:
# Path dan nama file
output_path = clean_path / "Data Final For Modeling.csv"

# Simpan ke CSV
merged_df.to_csv(output_path, index=False)

print(f"✅ Data berhasil disimpan ke: {output_path}")

✅ Data berhasil disimpan ke: ..\data\Data Clean\Data Final For Modeling.csv
